In [31]:
import pandas as pd
import networkx as nx
query_network = pd.read_csv('c:\\Users\\Alexander\\Downloads\\suspicious_edges.csv')
given_network = pd.read_csv('c:\\Users\Alexander\\Downloads\\social_network.csv')
ans = pd.read_csv('c:\\Users\\Alexander\\Downloads\\ans.csv')

Фиксим баг с именем `'j '`

In [32]:
given_network.columns = [u'i', u'j']
query_network.columns = [u'id', u'i', u'j']

Задел на будущее, опробуем SSVM

In [33]:
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Найдём петли в графе

In [34]:
[(i, j) for _, (i, j) in given_network.iterrows() if i == j]

[(1075, 1075),
 (1107, 1107),
 (1355, 1355),
 (1652, 1652),
 (2023, 2023),
 (2346, 2346),
 (2423, 2423),
 (3159, 3159),
 (3388, 3388),
 (4364, 4364),
 (6647, 6647),
 (7973, 7973),
 (7986, 7986),
 (8789, 8789),
 (9524, 9524)]

In [35]:
[(i, j) for _, (_, i, j) in query_network.iterrows() if i == j]

[(2881, 2881),
 (2183, 2183),
 (1249, 1249),
 (1234, 1234),
 (599, 599),
 (8196, 8196),
 (1036, 1036),
 (3775, 3775),
 (4735, 4735),
 (597, 597),
 (282, 282),
 (2634, 2634),
 (62, 62),
 (3920, 3920),
 (6980, 6980),
 (940, 940)]

Петель мало, так что ими можно пренебречь. Удалим их из выборки.

In [36]:
given_excluded = given_network[given_network.i == given_network.j]
given_network = given_network[given_network.i != given_network.j]
query_excluded = query_network[query_network.i == query_network.j]
query_network = query_network[query_network.i != query_network.j]

In [37]:
edges = [(i, j) for _, (i, j) in given_network.iterrows()]
query_edges = [(i, j) for _, (_, i, j) in query_network.iterrows()]

In [38]:
nodes = pd.Series(given_network[['i', 'j']].values.ravel()).unique()
query_nodes = pd.Series(query_network[['i', 'j']].values.ravel()).unique()

In [39]:
G = nx.Graph()
G.add_edges_from(edges)
G.add_nodes_from(query_nodes)

Для оценки вероятности используем  Resource allocation index

In [40]:
aa_idxs = nx.resource_allocation_index(G, query_edges)

In [41]:
probs = array([prob for (_, _, prob) in aa_idxs])
probs /= probs.max()

In [42]:
query_excluded['prob'] = 0
query_network['prob'] = probs

In [43]:
response_total = pd.concat([query_excluded, query_network])[['id', 'prob']].\
                                                                 sort_index()

In [44]:
with open('c:\\Users\\Alexander\\Downloads\\rai.csv', 'w') as fo:
    response_total.to_csv(fo, index=False)